In [1]:
import uuid
import time
import imghdr
import io
import os
import threading
from datetime import datetime
from queue import Queue

import pandas
import requests
from PIL import Image
from tqdm import tqdm

recordsQ = Queue()
resultQ = Queue()


def GetImg(url, name, folderName):
    response = requests.get(url, headers={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,image/*,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.34'})
    image_format = imghdr.what(None, h=response.content)
    if image_format is None:
        return None
    if image_format == 'webp':
        with Image.open(io.BytesIO(response.content)) as img:
            img = img.convert("RGB")
            img.save(f"{name}.jpg", "JPEG")
            return f"{name}.jpg"
    nName = f'{folderName}/{name}.{image_format}'
    with open(nName, 'wb') as f:
        f.write(response.content)

    return f'{name}.{image_format}'


folderName = 'Images'
if not os.path.exists(folderName):
    os.mkdir(folderName)

def GetUniqId(isTimeStamp=False):
    # Generate a UUID
    return str(int(time.time_ns() / 1_000_000)) if isTimeStamp else str(uuid.uuid4())

def Download(progress_bar, colName):
    while not recordsQ.empty():
        record_dic = recordsQ.get()
        hrefs = record_dic.get(colName).split('\n')
        for x, href in enumerate(hrefs):
            if not pandas.isna(href):
                name = f'{GetUniqId(True)}_{GetUniqId()}'
                try:
                    nName = GetImg(href, name,folderName)
                    record_dic[f'imgLocation_{x}'] = nName
                except Exception as e:
                    pass

        progress_bar.update(1)
        resultQ.put(record_dic)


i = 'kalamandirroyale Bridal Sarees_scrapped_result.csv'
path = f'{i}'

if i.__contains__('.csv'):
    data = pandas.read_csv(path)
else:
    data = pandas.read_excel(path)

image_column = 'images'
urls = data.to_dict('records')

[recordsQ.put(_) for _ in urls]
threads = []

pbar = tqdm(total=len(urls))

for worker in range(6):
    thread = threading.Thread(target=Download, args=(pbar, image_column))
    thread.start()
    threads.append(thread)

for t in threads:
    t.join()


def qToDf(q):
    l = []
    while not q.empty():
        l.append(q.get())
    [q.put(_) for _ in l]
    df = pandas.DataFrame(l)
    return df


qToDf(resultQ).to_csv(f'with_images_{i}', index=False)


100%|██████████| 174/174 [03:19<00:00,  3.76s/it]